In [1]:
library(SCENT)
library(Signac)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(parallel)

Warning message:
“replacing previous import ‘Hmisc::capitalize’ by ‘R.utils::capitalize’ when loading ‘SCENT’”
Loading required package: grid

Warning message:
“replacing previous import ‘Biostrings::pattern’ by ‘grid::pattern’ when loading ‘genomation’”
Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following object

Specify file path

In [2]:
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/3.Genome_wide_prediction/SCENT/SCENT_2500.240606/pairs.E2G.res.tsv.gz"
path.pairs.ABC = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/240508/sc-E2G/test/results_K562_Xu/K562/Predictions/EnhancerPredictionsAllPutative.tsv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/K562_Xu/3.Genome_wide_prediction/SCENT.default/SCENT.extract_2500.240622/"

In [3]:
n.cores = 16

Import candidate E-G pairs

In [4]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import ABC results

In [5]:
pairs.ABC = readGeneric(path.pairs.ABC,
                        header = T,
                        keep.all.metadata = T)

Filter ABC results which distance < 500kb

In [6]:
pairs.ABC.500kb = pairs.ABC[pairs.ABC$distance < 500000]

Filter E-G pairs overlaping with pairs.ABC.500kb

In [7]:
df.pairs.E2G.chr_rename = as.data.frame(pairs.E2G)[,1:3]
df.pairs.E2G.chr_rename[,"seqnames"] = paste(seqnames(pairs.E2G),
                                             mcols(pairs.E2G)[,"TargetGene"],
                                             sep = "_")
pairs.E2G.chr_rename = GRanges(df.pairs.E2G.chr_rename)
rm(df.pairs.E2G.chr_rename)

df.pairs.ABC.500kb.chr_rename = as.data.frame(pairs.ABC.500kb)[,1:3]
df.pairs.ABC.500kb.chr_rename[,"seqnames"] = paste(seqnames(pairs.ABC.500kb),
                                                mcols(pairs.ABC.500kb)[,"TargetGene"],
                                                sep = "_")
pairs.ABC.500kb.chr_rename = GRanges(df.pairs.ABC.500kb.chr_rename)
rm(df.pairs.ABC.500kb.chr_rename)

pairs.E2G.filter = pairs.E2G[countOverlaps(pairs.E2G.chr_rename,
                                           pairs.ABC.500kb.chr_rename) > 0]
rm(pairs.E2G.chr_rename)
rm(pairs.ABC.500kb.chr_rename)
pairs.E2G.filter

GRanges object with 102290 ranges and 10 metadata columns:
           seqnames              ranges strand |  TargetGene    CellType
              <Rle>           <IRanges>  <Rle> | <character> <character>
       [1]     chr1 100027917-100029695      * |         AGL        K562
       [2]     chr1 100027917-100029695      * |         DBT        K562
       [3]     chr1 100036824-100037384      * |     SLC35A3        K562
       [4]     chr1 100037554-100039259      * |      CDC14A        K562
       [5]     chr1     1001483-1002126      * |       CCNL2        K562
       ...      ...                 ...    ... .         ...         ...
  [102286]     chrX     9462775-9463861      * |       TBL1X        K562
  [102287]     chrX     9463954-9464413      * |       TBL1X        K562
  [102288]     chrX     9464499-9466458      * |       TBL1X        K562
  [102289]     chrX     9530658-9531443      * |       TBL1X        K562
  [102290]     chrX     9708493-9709582      * |       TBL1X     

In [8]:
dir.create(dir.output,recursive = T)

In [9]:
df.output = as.data.frame(pairs.E2G.filter,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = "K562"
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "beta","se","z","p","boot_basic_p",
                         "boot_basic_p.log10",
                         "sign",
                         "boot_basic_p.log10.signed")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
df.output

chr,start,end,TargetGene,CellType,beta,se,z,p,boot_basic_p,boot_basic_p.log10,sign,boot_basic_p.log10.signed
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>
chr1,100027917,100029695,AGL,K562,0.10575638,0.03941305,2.683283,7.290316e-03,0.0160,1.795880,1,1.795880
chr1,100027917,100029695,DBT,K562,0.11841151,0.04531618,2.613007,8.974945e-03,0.0080,2.096910,1,2.096910
chr1,100036824,100037384,SLC35A3,K562,0.11283328,0.03870542,2.915181,3.554827e-03,0.0048,2.318759,1,2.318759
chr1,100037554,100039259,CDC14A,K562,0.04563888,0.01153155,3.957739,7.566258e-05,0.0016,2.795880,1,2.795880
chr1,1001483,1002126,CCNL2,K562,0.09105191,0.01841712,4.943874,7.658530e-07,0.0008,3.096910,1,3.096910
chr1,1001483,1002126,HES4,K562,0.20783112,0.04300763,4.832424,1.348803e-06,0.0008,3.096910,1,3.096910
chr1,100246661,100247036,RTCA,K562,0.07386780,0.03376908,2.187439,2.871047e-02,0.0424,1.372634,1,1.372634
chr1,100248957,100250307,CDC14A,K562,0.02908796,0.01160134,2.507294,1.216594e-02,0.0472,1.326058,1,1.326058
chr1,100248957,100250307,SASS6,K562,0.05259358,0.02002335,2.626612,8.623962e-03,0.0256,1.591760,1,1.591760


In [10]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
[1] Matrix_1.6-5         GenomicRanges_1.54.1 GenomeInfoDb_1.38.1 
[4] IRanges_2.36.0       S4Vectors_0.40.2     BiocGeneri